In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv('../data/current.csv')
# df = pd.read_csv('../../training_data.csv')
df.head()
df.shape


(879, 22)

### Cleaning

In [3]:
df.drop(df['skaterFullName'].loc[df['skaterFullName'] == 'Taylor Hall'].index, inplace=True)
df = df.drop(columns=['skaterFullName', 'teamAbbrevs', 'vs', 'shoots', 'row', 'date']) 
df = df.dropna()
df.head()

,ppPoints,gamesPlayed,position,ppUnit,avgPowerplayToi,team_PEN/GP,team_PP%,team_PK%,team_G,team_GA,enemy_PEN/GP,enemy_PP%,enemy_PK%,enemy_G,enemy_GA,scored
0,0,9,L,2.0,1:39,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,0
1,0,10,D,1.0,1:58,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,1
2,3,10,L,2.0,3:38,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,0
3,4,10,R,1.0,3:46,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,1
4,0,10,C,2.0,1:07,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,1


In [4]:
df['avgPowerplayToi'] = df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)

df['position'].replace(['C', 'R', 'L', 'D'], [3, 2, 2, 1], inplace=True)
df.head()

,ppPoints,gamesPlayed,position,ppUnit,avgPowerplayToi,team_PEN/GP,team_PP%,team_PK%,team_G,team_GA,enemy_PEN/GP,enemy_PP%,enemy_PK%,enemy_G,enemy_GA,scored
0,0,9,2,2.0,99,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,0
1,0,10,1,1.0,118,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,1
2,3,10,2,2.0,218,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,0
3,4,10,2,1.0,226,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,1
4,0,10,3,2.0,67,4.5,15.6,97.4,31.0,16.0,4.73,28.6,83.3,40,33,1


### Resampling

### Neural Network

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.layers.experimental.preprocessing import Normalization


In [6]:
features = ['ppPoints', 'gamesPlayed', 'avgPowerplayToi', 'team_PEN/GP', 'team_PP%', 'team_PK%', 'team_GA', 'team_G', 'enemy_PEN/GP', 'enemy_PP%', 'enemy_PK%', 'enemy_GA', 'enemy_G']

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])
# scaler = StandardScaler()
# df[features] = scaler.fit_transform(df[features])

X = df[features].values
y = df['scored'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Fit and transform the training data using the RandomUnderSampler
# X_train, y_train = undersampler.fit_resample(X_train, y_train)
X_train, y_train = oversampler.fit_resample(X_train, y_train)

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

model = Sequential()
model.add(Dense(128))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train, epochs=150, batch_size=64, validation_data=(X_test, y_test), verbose=1, shuffle=True)

Epoch 1/150
17/17 [==============================] - 2s 24ms/step - loss: 0.6893 - precision: 0.5372 - val_loss: 0.7078 - val_precision: 0.2213
Epoch 2/150
17/17 [==============================] - 0s 4ms/step - loss: 0.6734 - precision: 0.6155 - val_loss: 0.6830 - val_precision: 0.2267
Epoch 3/150
17/17 [==============================] - 0s 4ms/step - loss: 0.6561 - precision: 0.6682 - val_loss: 0.7783 - val_precision: 0.2212
Epoch 4/150
17/17 [==============================] - 0s 3ms/step - loss: 0.6473 - precision: 0.6239 - val_loss: 0.7041 - val_precision: 0.2105
Epoch 5/150
17/17 [==============================] - 0s 4ms/step - loss: 0.6409 - precision: 0.6351 - val_loss: 0.7416 - val_precision: 0.2235
Epoch 6/150
17/17 [==============================] - 0s 4ms/step - loss: 0.6357 - precision: 0.6330 - val_loss: 0.6510 - val_precision: 0.2414
Epoch 7/150
17/17 [==============================] - 0s 4ms/step - loss: 0.6290 - precision: 0.6598 - val_loss: 0.8768 - val_precision: 0.221

In [7]:
from sklearn.metrics import classification_report


y_pred_probs = model.predict(X_test)

# Adjust the threshold
threshold = 0.95 # You can experiment with different threshold values
y_pred_labels = (y_pred_probs > threshold).astype(int)

print(f'threshold of {threshold*100}%')
print(classification_report(y_test, y_pred_labels))

y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
print('Regular threshold')
print(classification_report(y_test, y_pred))
# test_lost, test_acc = model.evaluate(X_test, y_test)
# print(f'Test Accuracy: {round(test_acc*100, 2)}%')

6/6 [==============================] - 0s 1ms/step
threshold of 95.0%
              precision    recall  f1-score   support

           0       0.81      0.99      0.89       135
           1       0.80      0.11      0.20        35

    accuracy                           0.81       170
   macro avg       0.81      0.55      0.55       170
weighted avg       0.81      0.81      0.75       170

6/6 [==============================] - 0s 1ms/step
Regular threshold
              precision    recall  f1-score   support

           0       0.88      0.75      0.81       135
           1       0.38      0.60      0.47        35

    accuracy                           0.72       170
   macro avg       0.63      0.67      0.64       170
weighted avg       0.78      0.72      0.74       170



## Player List Prediction

In [8]:
df = pd.read_csv('../../../lib/ai_bum_list.csv')
df.drop(df['skaterFullName'].loc[df['skaterFullName'] == 'Taylor Hall'].index, inplace=True)
df = df.drop(columns=['skaterFullName', 'teamAbbrevs', 'vs', 'shoots', 'row', 'date']) 
df = df.dropna()

df['avgPowerplayToi'] = df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)

df['position'].replace(['C', 'R', 'L', 'D'], [3, 2, 2, 1], inplace=True)
df.head()
df.shape

(188, 16)

In [12]:
list_df = pd.read_csv('../../../lib/ai_bum_list.csv')

list_df.drop(list_df['skaterFullName'].loc[list_df['skaterFullName'] == 'Taylor Hall'].index, inplace=True)
list_df['avgPowerplayToi'] = list_df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)

features = ['ppPoints', 'gamesPlayed', 'avgPowerplayToi', 'team_PEN/GP', 'team_PP%', 'team_PK%', 'team_GA', 'team_G', 'enemy_PEN/GP', 'enemy_PP%', 'enemy_PK%', 'enemy_GA', 'enemy_G']

# normalized_new_data = normalization_layer(list_df[features])

list_df[features] = scaler.fit_transform(list_df[features])

# Iterate over each row and reshape
reshaped_data_list = []
for index, row in list_df.iterrows():
    reshaped_data = row[features].values.reshape(1, 1, 13)
    reshaped_data_list.append(reshaped_data)


# Convert the list to a NumPy array
reshaped_data_array = np.array(reshaped_data_list, dtype=np.float32)

predictions = []
for i in reshaped_data_array:
    pred = model.predict(i)
    predictions.append(pred[0][0])


list_df['prediction'] = predictions
list_df.sort_values(by='prediction', ascending=False).to_csv('rnn_preds.csv')


1/1 [==============================] - 0s 12ms/step


## Single Player Prediction

In [10]:
# player = [{
#     'skaterFullName': 'Connor Bedard',
#     'ppPoints': 1,
#     'teamAbbrevs': 'CHI',
#     'gamesPlayed': 11,
#     'shoots': 'R',
#     'position': 'C',
#     'ppUnit': 1,
#     'vs': 'TB',
#     'avgPowerplayToi': '2:45',
#     'row': 0,
#     'team_PEN/GP': 4.27,
#     'team_PP%': 10.5,
#     'team_PK%': 83.3,
#     'team_G': 26,
#     'team_GA': 38,
#     'enemy_PEN/GP': 4.08,
#     'enemy_PP%': 34.1,
#     'enemy_PK%': 87.2,
#     'enemy_G': 50,
#     'enemy_GA': 45,
#     'scored': 1
# }]

# player_df = pd.DataFrame(player)

# player_df['avgPowerplayToi'] = player_df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)

# features = ['ppPoints', 'gamesPlayed', 'avgPowerplayToi', 'team_PEN/GP', 'team_PP%', 'team_PK%', 'team_GA', 'team_G', 'enemy_PEN/GP', 'enemy_PP%', 'enemy_PK%', 'enemy_GA', 'enemy_G']

# player_df[features] = scaler.fit_transform(player_df[features])

# # Reshape the player to fit the LSTM model
# prediction = model.predict(player_df[features].values.reshape(1, 1, 13))

# prediction


In [11]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

KeyError: 'accuracy'